###

We'll evaluate if all sources have same columns, then we'll use pandas for our cleansing. We'll import re module in case on further analysis we need to focus on bra size.

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
ae = pd.read_csv('src/ae_com.csv')
amz = pd.read_csv('src/amazon_com.csv')
bte = pd.read_csv('src/btemptd_com.csv')
ck = pd.read_csv('src/calvinklein_com.csv')
hp = pd.read_csv('src/hankypanky_com.csv')
macys = pd.read_csv('src/macys_com.csv')
ns = pd.read_csv('src/shop_nordstrom_com.csv')
tops = pd.read_csv('src/us_topshop_com.csv')
vs = pd.read_csv('src/victoriassecret_com.csv')

In [3]:
frames = [ae,amz,bte,ck,hp,macys,ns,tops,vs]

In [4]:
for frame in frames:
    print(frame.shape)

(1590, 13)
(1452, 13)
(608, 13)
(737, 13)
(3183, 13)
(3409, 13)
(1196, 13)
(419, 13)
(6077, 13)


In [5]:
transactions = pd.concat(frames)

In [6]:
transactions.groupby(by='retailer')['retailer'].count()

retailer
Ae US                 1590
Amazon US             1452
Btemptd US             608
Calvin Klein US        737
Hankypanky US         3183
Macys US              3409
Nordstrom US          1196
Topshop US             419
Victoriassecret US    6077
Name: retailer, dtype: int64

In [7]:
transactions.shape

(18671, 13)

In [8]:
transactions.head()

,product_name,mrp,pdp_url,color,total_sizes,available_size,brand_name,product_category,retailer,description,rating,review_count,scrapping_datetime
0,Aerie Adjustable Low-Back Strap,15.5,Aerie Adjustable Low-Back Strap,Multi,One Size,One Size,AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,NaN,NaN,25-04-20 13:25
1,Aerie Adjustable Low-Back Strap,15.5,Aerie Adjustable Low-Back Strap,Multi,One Size,One Size,AERIE,Strapless Bras,Ae US,Quick FixesÂ® Collection: Endless options for ...,4.0,NaN,25-04-20 13:25
2,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,"A, B, C, D","A, B, C, D",AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25
3,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,"A, B, C, D","A, B, C, D",AERIE,Strapless Bras,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25
4,Aerie Backless Pushup Bare Bra,38.5,Aerie Backless Pushup Bare Bra,Dark Nude,"A, B, C, D","A, B, C, D",AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,NaN,NaN,25-04-20 13:25


We save the dataframe for any future issue.

In [9]:
transactions.to_csv('src/transactions.csv')

### Data Cleansing for size information
We'll generate a transactions table with each size and its availability.
The following function will count the existence per size of each model

In [10]:
def count_available_sizes(totals, avail):
    totals = totals.replace(" ", "").split(",")
    avail = avail.replace(" ", "").split(",")
    output = [(size, int(size in avail)) for size in totals]
    return output

This function generates a list of tuples, each of them consisting of the size and 1 or 0 to indicate if that size is available.
We generate the new columns, drop total_size and available_size.
After this, we'll explode the list so we can have a unique transaction for the availability of the product per size.

In [11]:
transactions['all_sizes'] = transactions.apply(lambda x: count_available_sizes(x['total_sizes'], x['available_size']), axis=1)

In [12]:
transactions

,product_name,mrp,pdp_url,color,total_sizes,available_size,brand_name,product_category,retailer,description,rating,review_count,scrapping_datetime,all_sizes
0,Aerie Adjustable Low-Back Strap,15.5,Aerie Adjustable Low-Back Strap,Multi,One Size,One Size,AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,NaN,NaN,25-04-20 13:25,"[(OneSize, 1)]"
1,Aerie Adjustable Low-Back Strap,15.5,Aerie Adjustable Low-Back Strap,Multi,One Size,One Size,AERIE,Strapless Bras,Ae US,Quick FixesÂ® Collection: Endless options for ...,4.0,NaN,25-04-20 13:25,"[(OneSize, 1)]"
2,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,"A, B, C, D","A, B, C, D",AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25,"[(A, 1), (B, 1), (C, 1), (D, 1)]"
3,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,"A, B, C, D","A, B, C, D",AERIE,Strapless Bras,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25,"[(A, 1), (B, 1), (C, 1), (D, 1)]"
4,Aerie Backless Pushup Bare Bra,38.5,Aerie Backless Pushup Bare Bra,Dark Nude,"A, B, C, D","A, B, C, D",AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,NaN,NaN,25-04-20 13:25,"[(A, 1), (B, 1), (C, 1), (D, 1)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6072,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,warm geo print,"XS, S, M, L",XS,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,"[(XS, 1), (S, 0), (M, 0), (L, 0)]"
6073,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,cocoon,"XS, S, M, L",XS,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,"[(XS, 1), (S, 0), (M, 0), (L, 0)]"
6074,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,midnight tropical,"XS, S, M, L",XS,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,"[(XS, 1), (S, 0), (M, 0), (L, 0)]"
6075,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,cool geo print,"XS, S, M, L",XS,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,"[(XS, 1), (S, 0), (M, 0), (L, 0)]"


Then we can treat each size and its availability as a transaction.

In [13]:
transactions = transactions.explode('all_sizes')

In [14]:
transactions

,product_name,mrp,pdp_url,color,total_sizes,available_size,brand_name,product_category,retailer,description,rating,review_count,scrapping_datetime,all_sizes
0,Aerie Adjustable Low-Back Strap,15.5,Aerie Adjustable Low-Back Strap,Multi,One Size,One Size,AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,NaN,NaN,25-04-20 13:25,"(OneSize, 1)"
1,Aerie Adjustable Low-Back Strap,15.5,Aerie Adjustable Low-Back Strap,Multi,One Size,One Size,AERIE,Strapless Bras,Ae US,Quick FixesÂ® Collection: Endless options for ...,4.0,NaN,25-04-20 13:25,"(OneSize, 1)"
2,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,"A, B, C, D","A, B, C, D",AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25,"(A, 1)"
2,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,"A, B, C, D","A, B, C, D",AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25,"(B, 1)"
2,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,"A, B, C, D","A, B, C, D",AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25,"(C, 1)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6075,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,cool geo print,"XS, S, M, L",XS,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,"(L, 0)"
6076,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,moonray,"XS, S, M, L",XS,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,"(XS, 1)"
6076,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,moonray,"XS, S, M, L",XS,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,"(S, 0)"
6076,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,moonray,"XS, S, M, L",XS,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,"(M, 0)"


We then proceed to split the tuple into size (dimension) and availability (metric)

In [15]:
transactions[['size', 'availability']] = transactions['all_sizes'].apply(lambda x: pd.Series({'size': x[0], 'availability': x[1]}))

In [16]:
transactions

,product_name,mrp,pdp_url,color,total_sizes,available_size,brand_name,product_category,retailer,description,rating,review_count,scrapping_datetime,all_sizes,size,availability
0,Aerie Adjustable Low-Back Strap,15.5,Aerie Adjustable Low-Back Strap,Multi,One Size,One Size,AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,NaN,NaN,25-04-20 13:25,"(OneSize, 1)",OneSize,1
1,Aerie Adjustable Low-Back Strap,15.5,Aerie Adjustable Low-Back Strap,Multi,One Size,One Size,AERIE,Strapless Bras,Ae US,Quick FixesÂ® Collection: Endless options for ...,4.0,NaN,25-04-20 13:25,"(OneSize, 1)",OneSize,1
2,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,"A, B, C, D","A, B, C, D",AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25,"(A, 1)",A,1
2,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,"A, B, C, D","A, B, C, D",AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25,"(B, 1)",B,1
2,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,"A, B, C, D","A, B, C, D",AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25,"(C, 1)",C,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6075,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,cool geo print,"XS, S, M, L",XS,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,"(L, 0)",L,0
6076,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,moonray,"XS, S, M, L",XS,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,"(XS, 1)",XS,1
6076,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,moonray,"XS, S, M, L",XS,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,"(S, 0)",S,0
6076,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,moonray,"XS, S, M, L",XS,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,"(M, 0)",M,0


We can drop now the columns that we're not using anymore:
1. total_sizes
2. available_size
3. all_sizes

In [18]:
transactions = transactions.drop(columns=['total_sizes','available_size','all_sizes'])

KeyError: "['total_sizes', 'available_size', 'all_sizes'] not found in axis"

In [20]:
transactions.columns

Index(['product_name', 'mrp', 'pdp_url', 'color', 'brand_name',
       'product_category', 'retailer', 'description', 'rating', 'review_count',
       'scrapping_datetime', 'size', 'availability'],
      dtype='object')

In [21]:
transactions.shape

(176693, 13)

Now we can add the size group it belongs to; first we'll focus on the rules shared by business.

1. The type of innerwear may be identified for the structure of the size:
  * [S, M, L, XL, XXL,...] are not bras. These sizes are related to bikinis, panties or tops.
  * [6,7,8,9,...] are not bras. They are related to boypants, briefs, thongs, knickers, and other below-navel items.
  * [A,B,C,D,...] are not considered bras. These sizes are related to adhesive silicone bras.
  * [1X, 2X, 3X,...] are considered bras. They include body, dresses and chest-related items for big sizes.
  * [34A, 36B, 40D, 36A/B, 36G(4D),...] are considered bras. These are the most common size structures for bras in which the number describes the underbust size and the letter is related to the bust size.
2. According to the size, the bras are categorized into 4 size groups:
  * “Small” = Underbust sizes in the group: [30,32]
  * “Medium” = Underbust sizes in the group: [34,36]
  * “Large” = Underbust sizes in the group: [38,40]
  * “Extra Large” = Underbust sizes in the group: [42,44,46] and sizes with the structure [1X, 2X, 3X,...]

We'll generate with regex the column that will handle the size group for our bras, since we don't have a lot of detail for the other categories we'll fill them with null values.
After that we'll create a new column called "main_category" to divide 'bra' and 'others'. And after that we'll assign the sizes for the other items.

In [22]:
def size_group(size):
    size = str(size)
    if re.search(r'^[1-9]X',size):
        return 'Extra Large'
    elif re.search(r'\d{2}[A-Z]',size) or re.search(r'\d{2}[A-Z]\(\d+[A-Z]{2}\)',size):
        num_part = int(size[:2])
        if num_part in [30, 32]:
            return 'Small'
        elif num_part in [34, 36]:
            return 'Medium'
        elif num_part in [38, 40]:
            return 'Large'
        elif num_part in [42, 44, 46]:
            return 'Extra Large'
    else:
        return None

In [23]:
transactions['size_group'] = transactions.apply(lambda x: size_group(x['size']), axis=1)

In [24]:
transactions.head(4)

,product_name,mrp,pdp_url,color,brand_name,product_category,retailer,description,rating,review_count,scrapping_datetime,size,availability,size_group
0,Aerie Adjustable Low-Back Strap,15.5,Aerie Adjustable Low-Back Strap,Multi,AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,NaN,NaN,25-04-20 13:25,OneSize,1,None
1,Aerie Adjustable Low-Back Strap,15.5,Aerie Adjustable Low-Back Strap,Multi,AERIE,Strapless Bras,Ae US,Quick FixesÂ® Collection: Endless options for ...,4.0,NaN,25-04-20 13:25,OneSize,1,None
2,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25,A,1,None
2,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25,B,1,None


In [25]:
transactions['main_category'] = transactions['size_group'].apply(lambda x: 'Others' if pd.isna(x) else 'Bra')

In [26]:
transactions

,product_name,mrp,pdp_url,color,brand_name,product_category,retailer,description,rating,review_count,scrapping_datetime,size,availability,size_group,main_category
0,Aerie Adjustable Low-Back Strap,15.5,Aerie Adjustable Low-Back Strap,Multi,AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,NaN,NaN,25-04-20 13:25,OneSize,1,None,Others
1,Aerie Adjustable Low-Back Strap,15.5,Aerie Adjustable Low-Back Strap,Multi,AERIE,Strapless Bras,Ae US,Quick FixesÂ® Collection: Endless options for ...,4.0,NaN,25-04-20 13:25,OneSize,1,None,Others
2,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25,A,1,None,Others
2,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25,B,1,None,Others
2,Aerie Backless Bare Bra,36.5,Aerie Backless Bare Bra,Dark Nude,AERIE,Bra Accessories,Ae US,Quick FixesÂ® Collection: Endless options for ...,5.0,NaN,25-04-20 13:25,C,1,None,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6075,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,cool geo print,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,L,0,None,Others
6076,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,moonray,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,XS,1,None,Others
6076,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,moonray,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,S,0,None,Others
6076,PINK NEW! Lace Lightly Lined Halter Bralette,474.522,PINK NEW! Lace Lightly Lined Halter Bralette,moonray,Victoria's Secret Pink,Lace Lightly Lined Halter Bralette,Victoriassecret US,"Made for layering and lounging, this halter br...",4.4,74.0,26-04-20 19:57,M,0,None,Others


In [27]:
transactions.to_csv('src/transactions.csv')

We perform the check on rating to see if it's between 0 and 5

In [28]:
transactions.groupby(by='rating')['rating'].count()

rating
1.0      82
1.5       6
1.7       6
2.0     313
2.4      60
2.5      93
2.7       7
3.0     461
3.1     112
3.2     448
3.3     234
3.4      30
3.5     266
3.6     792
3.7     698
3.8     841
3.9     508
4.0    1913
4.1     525
4.2    1375
4.3    1391
4.4    1409
4.5    3635
4.6    5186
4.7    2218
4.8    1190
4.9     739
5.0    5933
Name: rating, dtype: int64

Checked ✅ the type is int64 and the minimum and maximum values are between 1 and 5.
We proceed with reviews.

In [35]:
transactions.groupby(by='review_count')['review_count'].count()

review_count
1.000000e+00    2918
2.000000e+00    2212
3.000000e+00    1005
4.000000e+00     816
5.000000e+00     980
                ... 
5.070000e+02     360
5.180000e+02     168
5.540000e+02     220
6.420000e+02      60
1.560000e+38      10
Name: review_count, Length: 105, dtype: int64

In [36]:
def clean_review_count(val):
    if pd.isna(val):
        return np.nan
    else:
        int(val)
    if val < 10000:
        return int(val)
    else:
        return np.nan

In [37]:
transactions['review_count'] = transactions.apply(lambda x: clean_review_count(x['review_count']), axis=1)

In [38]:
transactions.groupby(by='review_count')['review_count'].count()

review_count
1.0      2918
2.0      2212
3.0      1005
4.0       816
5.0       980
         ... 
476.0      30
507.0     360
518.0     168
554.0     220
642.0      60
Name: review_count, Length: 104, dtype: int64

Finally we check for mrp

In [41]:
transactions.groupby(by='mrp')['mrp'].count()

mrp
7.5       157
7.51        4
7.99        5
8.0        52
8.5       725
         ... 
$82.00     16
$84.00      1
$89.00      9
$96.00      2
$98.00      6
Name: mrp, Length: 721, dtype: int64

As we have values inputed as string, we need to convert all to float in order to perform proper insights with it.

In [46]:
def convert_to_float(value):
    if isinstance(value, str) and '$' in value:
        return float(value.strip('$').replace(',', '').replace('.', '').replace(' ', ''))
    else:
        return float(value)

In [47]:
transactions['mrp'] = transactions.apply(lambda x: convert_to_float(x['mrp']), axis=1)

In [48]:
transactions.groupby(by='mrp')['mrp'].count()

mrp
7.50        157
7.51          4
7.99          5
8.00         52
8.50        725
           ... 
8900.00       9
9600.00       2
9800.00       6
10200.00      6
13500.00      8
Name: mrp, Length: 721, dtype: int64